In [ ]:
!pip install fake_useragent

In [ ]:
import requests
import time
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import os
import json
import zipfile

In [ ]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

def scrape_articles(max_pages=12):
    """
    Функция для получения ссылок статей с сайта https://habr.com/ru/hubs/programming/articles/

    Args:
        max_pages (int): Максимальное количество страниц для обработки.

    Returns:
        list: Список ссылок на статьи.
    """
    article_links = []
    ua = UserAgent()

    for page_num in range(1, max_pages + 1):
        url = f'https://habr.com/ru/hubs/programming/articles/page{page_num}/'
        session = requests.session()

        try:
            req = session.get(url, headers={'User-Agent': ua.random})
            req.raise_for_status()

            page = req.text
            soup = BeautifulSoup(page, 'html.parser')

            article_blocks = soup.find_all('article', {'class': 'tm-articles-list__item'})
            for article_block in article_blocks:
                article_tag = article_block.find("a", class_="tm-title__link")
                article_link = "https://habr.com/" + article_tag["href"]
                article_links.append(article_link)

        except (requests.exceptions.RequestException, Exception) as e:
            print(f"Error occurred while scraping page {page_num}: {e}")

    return article_links


In [ ]:
import os
import time
import json
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

def parse_article_info(article_links):
    """
    Функция для получения информации о статье по ее ссылке и сохранения данных в текстовый файл.

    Args:
        article_links (list): Список ссылок на статьи.
    """
    ua = UserAgent()

    # Создаем папку для сохранения файлов
    os.makedirs("corpus", exist_ok=True)

    for link in article_links:
        session = requests.session()
        try:
            req = session.get(link, headers={'User-Agent': ua.random})
            req.raise_for_status()

            page = req.text
            soup = BeautifulSoup(page, 'html.parser')

            # Получение названия статьи
            article_title = soup.find("title").text.strip()

            # Получение ссылки на статью
            article_link = link

            # Получение даты публикации
            article_date = soup.find("time")["datetime"]

            # Получение ника автора
            if soup.find("a", class_="tm-user-info__username"):
                article_author = soup.find("a", class_="tm-user-info__username").text.strip()
            else:
                article_author = "Автор неизвестен"

            # Получение текста статьи
            article_content = soup.find("div", class_="article-formatted-body article-formatted-body article-formatted-body_version-2")

            metadata = {
                "author": article_author,
                "title": article_title,
                "link": article_link
            }

            if article_content:
                article_text = [p.text for p in article_content.find_all("p")]

                # Сохранение данных в текстовый файл
                path = os.path.join("corpus", article_link.split('/')[-2] + ".txt")
                with open(path, "w+", encoding="utf-8") as file:
                    file.write(json.dumps(metadata, ensure_ascii=False) + "\n\n")
                    file.write("\n".join(article_text))

            # Задержка, чтобы не перегружать сайт
            time.sleep(1)

        except (requests.exceptions.RequestException, Exception) as e:
            print(f"Error occurred while parsing article {link}: {e}")


In [ ]:
article_links = scrape_articles()
article_data = parse_article_info(article_links)

In [ ]:
source_folder = "corpus"
archive_name = "corpus.zip"

# Создаем архив
with zipfile.ZipFile(archive_name, "w") as zip_file:
    for root, dirs, files in os.walk(source_folder):
        for file in files:
            file_path = os.path.join(root, file)
            zip_file.write(file_path)
